In [17]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
import pandas as pd
import time
import os
import shutil
import unittest

In [2]:
def getHTML(url):
    driver = webdriver.Chrome(service =Service(ChromeDriverManager().install()))
    driver.get(url)
    with open("gdpPage.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)

In [3]:
# url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita"
# getHTML(url)

In [4]:
# pageFile =  open("gdpPage.html", encoding="utf-8")
# gdpPage = pageFile.read()
# soup = BeautifulSoup(gdpPage,'html.parser')

In [6]:
# table = soup.find_all("table")
# for t in table:
#     print(t.get("class"))

None
['wikitable', 'sortable', 'static-row-numbers', 'plainrowheaders', 'srn-white-background']
['nowraplinks', 'mw-collapsible', 'uncollapsed', 'navbox-inner']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'autocollapse', 'navbox-inner']


In [7]:
def getTable(htmlFile):
    driver = webdriver.Chrome(service =Service(ChromeDriverManager().install()))
    driver.get(htmlFile)
    gdpTable = driver.find_element(By.XPATH, "//*[@id='mw-content-text']/div[1]/table")
    print(driver.page_source)   

In [8]:
def cleanCountryName(text):
    return text[:len(text) - 2]

In [9]:
def putInEstimate(curRow, infoDictionary, agency, colIndex):
    """
        Put GDP estimate and year from the agency into the corresponding info dictionary.
        If there was no estimate for that agency, insert (0, 0).
        Arguments:
            curRow: The current <tr> element
            infoDictionary: dictionary to put the information into
            agency: "IMF", "UN", or "World Bank"
            colIndex: the current column
        return the column index - will be colIndex + 1 if no estimate, +2 if there was an estimate
    """
    keyToPutInfo = agency + " Estimate"
    
    if curRow[colIndex].has_attr('colspan'):
        infoDictionary[keyToPutInfo] = 0
        infoDictionary[agency + " Year"] = 0
        return colIndex + 1
    else:
        infoDictionary[keyToPutInfo] = curRow[colIndex].text
        infoDictionary[agency + " Year"] = curRow[colIndex + 1].text.split()[0]
        return colIndex + 2

In [25]:
pageFile =  open("gdpPage.html", encoding="utf-8")
gdpPage = pageFile.read()
soup = BeautifulSoup(gdpPage,'html.parser')


def createCountryDF():
    """
        Create and return the Wikipedia GDP Per Capita table as a pandas dataframe. 
    """
    pageFile =  open("gdpPage.html", encoding="utf-8")
    gdpPage = pageFile.read()
    soup = BeautifulSoup(gdpPage,'html.parser')
    gdpTable = soup.find("table", class_="wikitable sortable static-row-numbers plainrowheaders srn-white-background")
    gdpTableData = gdpTable.find("tbody")
    gdpRows = gdpTableData.find_all("tr")
    columns = ["Country/Territory", "UN Region", "IMF Estimate", " IMF Year", "UN Estimate", "UN Year",
           "World Bank Estimate", " World Bank Year" ]
    countryDF = pd.DataFrame(columns=columns)

    pdRow = 0;

    for i in range(2,len(gdpRows)):
        infoDictionary = {}

        r = gdpRows[i].find_all(["td"])
        if (len(r) != 0):
            infoDictionary['Country/Territory'] = cleanCountryName(r[0].find("a").text)
            infoDictionary['UN Region'] = r[1].text
            colIndex = 2
            agencies = ["IMF", "UN", "World Bank"]
            for agency in agencies:
                colIndex = putInEstimate(r, infoDictionary, agency, colIndex)
            countryDF.loc[len(countryDF.index)] = [infoDictionary[k] for k, v in infoDictionary.items()]
    return countryDF          

In [30]:
gdpPerCapitaDF = createCountryDF()
gdpPerCapitaDF.to_excel(os.path.join(os.getcwd(), "Data", "GDP Per Capita.xlsx"))


In [29]:
class TestGDPPerCapita(unittest.TestCase):
    
    def test_createCountryDF(self):
        df = createCountryDF()
        self.assertEqual(df.iloc[12]["UN Estimate"], "63,644")
unittest.main(argv=[''], verbosity=2, exit=False)
        

test_createCountryDF (__main__.TestGDPPerCapita) ... C:\Users\14159\AppData\Local\Temp\ipykernel_33728\41093741.py:4: ResourceWarning: unclosed file <_io.TextIOWrapper name='gdpPage.html' mode='r' encoding='utf-8'>
  df = createCountryDF()
ok

----------------------------------------------------------------------
Ran 1 test in 1.187s

OK
